In [17]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console
import os

In [18]:
# Configure Gemini using OpenAI compatible client
model = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    model_info={
        "vision": True,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
    },
)

clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""You are an expert editor focused on clarity and simplicity. 
    Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear. 
    Don't worry about persuasion or tone — just make the message easy to read and understand.""",
)

tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""You are a communication coach focused on emotional tone and professionalism. 
    Your job is to make the email sound warm, confident, and human — while staying professional 
    and appropriate for the audience. Improve the emotional resonance, polish the phrasing, 
    and adjust any words that may come off as stiff, cold, or overly casual.""",
)

persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""You are a persuasion expert trained in marketing, behavioral psychology, 
    and copywriting. Your job is to enhance the email's persuasive power: improve call to action, structure arguments, and emphasize benefits. Remove weak or passive language.""",
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""You are an advanced email-writing specialist. Your role is to read all 
    prior agent responses and revisions, and then **synthesize the best ideas** into a unified, 
    polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements; 
    Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional, 
    effective, and readable.""",
)

critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""You are an email quality evaluator. Your job is to perform a final review 
    of the synthesized email. Check for any remaining errors, awkward phrasing, or inconsistencies. 
    If the email is perfect, type 'APPROVE'. If not, provide specific feedback for improvement.""",
)

In [19]:
termination_condition = MaxMessageTermination(max_messages=10) | TextMentionTermination("APPROVE")

team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)

await Console(
    team.run_stream(
        task="Hi! Im hungry, buy me lunch and invest in my business. Thanks."
    )
)

---------- TextMessage (user) ----------
Hi! Im hungry, buy me lunch and invest in my business. Thanks.
---------- TextMessage (ClarityAgent) ----------
Buy me lunch and invest in my business.
---------- TextMessage (ToneAgent) ----------
This original email is a masterclass in how *not* to approach potential investors! It's demanding, lacks professionalism, context, and respect for the recipient's time.

Let's transform it into something that is warm, confident, human, and professional, suitable for seeking investment.

Here's an improved version, with explanations for the changes:

***

**Subject: An Exciting Opportunity in [Your Industry/Niche] – [Your Company Name]**

Dear [Investor's Name],

I'm writing to you today as the founder of **[Your Company Name]**, an innovative venture poised to [briefly state your compelling mission or impact, e.g., "revolutionize the way people connect," or "solve a critical problem in X industry"].

I've long admired your strategic investments at [In

TaskResult(messages=[TextMessage(id='6d062c53-fa61-458f-a325-03530b802dd8', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 25, 13, 56, 4, 37156, tzinfo=datetime.timezone.utc), content='Hi! Im hungry, buy me lunch and invest in my business. Thanks.', type='TextMessage'), TextMessage(id='5633c375-5385-4617-9c53-848bc97b5ab6', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=71, completion_tokens=9), metadata={}, created_at=datetime.datetime(2025, 11, 25, 13, 56, 7, 500237, tzinfo=datetime.timezone.utc), content='Buy me lunch and invest in my business.', type='TextMessage'), TextMessage(id='3e281486-025d-469f-86e0-8461f177f7b5', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=99, completion_tokens=838), metadata={}, created_at=datetime.datetime(2025, 11, 25, 13, 56, 22, 481261, tzinfo=datetime.timezone.utc), content='This original email is a masterclass in how *not* to approach potential investors! It\'s demanding, lacks pro